# Advanced Data Analysis

In [ ]:
import pandas as pd
lis_df = pd.read_parquet('data/processed/03_listings.parquet') 
rev_df = pd.read_parquet('data/processed/03_reviews.parquet')
print("Listings shape:", lis_df.shape)
print("Reviews shape:", rev_df.shape)

In [ ]:
lis_df.info()

In [ ]:
#TODO in SQL
# Response Quality: encode ordinal rating of each possible response time category
# lis_df['host_response_time']

# How many listings a host own (listings_count)

___

In [13]:
rev_df = pd.read_parquet('data/processed/03_reviews.parquet')
rev_s = pd.read_parquet('data/processed/04_sentiment_bert.parquet')

In [ ]:
rev_df.sort_values('listing_id')

In [ ]:
rev_s.sort_values('listing_id')

___

In [ ]:
# Find all columns containing 'host_listings' or matching 'host_total_listings_count'
host_listing_cols = [col for col in lis_df.columns if 'host_listings' in col or col == 'host_total_listings_count']
print("Columns containing 'host_listings' or 'host_total_listings_count':")
print(host_listing_cols)

In [ ]:
lis_df[host_listing_cols][:5]

In [ ]:
# Compare the three different host listing count columns
listing_counts = lis_df[['host_listings_count', 'host_total_listings_count', 'calculated_host_listings_count']]

# Find cases where they are all equal
all_equal = (listing_counts['host_listings_count'] == listing_counts['host_total_listings_count']) & \
            (listing_counts['host_total_listings_count'] == listing_counts['calculated_host_listings_count'])

print("Cases where all counts are equal:", sum(all_equal))
print("\nCases where counts differ:")
print(listing_counts[~all_equal].head())
print(f"\nTotal cases where counts differ: {sum(~all_equal)}")

# Check if host_total_listings_count is always the largest
is_largest = (listing_counts['host_total_listings_count'] >= listing_counts['host_listings_count']) & \
             (listing_counts['host_total_listings_count'] >= listing_counts['calculated_host_listings_count'])
print(f"\nIs host_total_listings_count always the largest? {is_largest.all()}")

if not is_largest.all():
    print("\nCases where host_total_listings_count is not the largest:")
    print(listing_counts[~is_largest].head())


___

In [ ]:
# Verify host_response_rate_pct is always 0 when host_response_time is empty
empty_response_time = lis_df[lis_df['host_response_time'] == '']
print("\nHost response rates when response time is empty:")
print(empty_response_time['host_response_rate_pct'].value_counts())
print(f"\nAll zeros? {(empty_response_time['host_response_rate_pct'] == 0).all()}")

# Remap empty strings to 'never' where host_response_rate_pct is 0
lis_df.loc[(lis_df['host_response_time'] == '') & (lis_df['host_response_rate_pct'] == 0), 'host_response_time'] = 'never'

lis_df['host_response_time'].unique()

Various types of analyses to understand the data better and to prepare for the database design.

In [ ]:
price_cols = [col for col in lis_df.columns if col.startswith('yearly')]
print("Price-related columns:", price_cols)

In [ ]:
lis_df['days_since_last_review']

In [ ]:
review_cols = ['total_reviews', 'number_of_reviews', 'reviews_per_month', 'number_of_reviews_l30d', 'yearly_review', 'number_of_reviews_ltm']
print("Review-related columns from listings:")
print(lis_df[review_cols].head())